In [1]:
# indir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/somatic_variants/needle/peaks_all/"
# outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/somatic_variants/needle/peaks_all/som_dendro_0.6/clones_variants_summary"
# cells_meta_f = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/cells_meta.tsv"
# dendro_indir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/barcodes/btwnClones_dendro_dt_0.6"
# to_chip = False
# chip_genes_f = "/data/Mito_Trace/output/geneRegions/gff_A2/chip_genes.bed"

dendro_indir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/barcodes/btwnClones_dendro_dt_0.6"
cells_meta_f = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/cells_meta.tsv"
indir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/somatic_variants/gatk_mutect/peaks_atac_peaks" 
outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/somatic_variants/gatk_mutect/peaks_atac_peaks/som_dendro_0.6/chip_clones_variants_summary"
to_chip = True 
chip_genes_f = "/data/Mito_Trace/output/geneRegions/gff_A2/chip_genes.bed" 
n_cpu =16

In [2]:
from src.utils.data_io import read_csv_multichar
import mplh.cluster_help as ch
import mplh.fig_utils as fu

fig_utils from mplh


In [3]:
from glob import glob
from os.path import basename, join, dirname, exists
import pandas as pd
from pandarallel import pandarallel
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
%load_ext autoreload
%autoreload 2
import numpy as np

In [4]:
pandarallel.initialize(nb_workers=n_cpu)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
import src.utils.dendro_clust as dc

In [6]:
#outdir = join(indir, "som_dendro")

In [7]:
cells_meta = pd.read_csv(cells_meta_f,sep="\t").set_index("ID")
cells_meta["name"] = cells_meta.apply(lambda x: f"{x['donor']}_{x['lineage']}", axis=1)
donors = cells_meta["donor"].unique()
conditions = cells_meta["condition"].unique()


cells_meta.head()


,lineage,donor,condition,name
ID,,,,
AAACGAAAGAATCAAC-1_Control,0,0,Control,0_0
AAATGAGAGAGACTCG-1_Control,0,0,Control,0_0
AACAAAGTCAAGTTGC-1_Control,0,0,Control,0_0
AACAAAGTCACATCCC-1_Control,0,0,Control,0_0
AACCAACGTTCATTTC-1_Control,0,0,Control,0_0


In [8]:
if to_chip:
    chip_df = pd.read_csv(chip_genes_f,sep="\t", header=None)
    cols = chip_df.columns.astype("object").values 
    cols[:6] = ["chr", "start", "end", "name","score","strand"]
    chip_df.columns = cols
    print(chip_df)
    

      chr      start        end name score strand       6     7  8  \
0    chr1   43332848   43357772    .     .      +  HAVANA  gene  .   
1   chr11  119201275  119318926    .     .      +  HAVANA  gene  .   
2   chr15   90078044   90107504    .     .      -  HAVANA  gene  .   
3   chr17    7656778    7692550    .     .      -  HAVANA  gene  .   
4   chr17   76729114   76742374    .     .      -  HAVANA  gene  .   
5    chr2   25222854   25347590    .     .      -  HAVANA  gene  .   
6    chr2  197383514  197440091    .     .      -  HAVANA  gene  .   
7    chr2  208231226  208271074    .     .      -  HAVANA  gene  .   
8   chr20   32353329   32444319    .     .      +  HAVANA  gene  .   
9   chr21   43087955   43112587    .     .      -  HAVANA  gene  .   
10   chr4  105140874  105284816    .     .      +  HAVANA  gene  .   
11   chr7  148802382  148889321    .     .      -  HAVANA  gene  .   
12   chr8  116840934  116879866    .     .      -  HAVANA  gene  .   

                   

## Load the somatic variants number of cells and merge across conditions

## Load dendro labels and barcodes mean af

### Merge the variants as well

In [9]:
def is_chip_gene(chip_gene, curr_var ):
    if (curr_var["chr"] == chip_gene["chr"]):
        if (chip_gene["strand"] == "+"):
            if curr_var["pos"]>=chip_gene["start"] & curr_var["pos"]<=chip_gene["end"]:
                return True    
        elif (chip_gene["strand"] == "-"):
            if curr_var["pos"]<=chip_gene["start"] & curr_var["pos"]>=chip_gene["end"]:
                return True
        else: 
            print("No strand on gene. Returning False")
    return False

def is_in_any_chip_gene(curr_var, chip_bed):
    for ind, curr_gene in chip_bed.iterrows():
        #print('curr_gene')
        #print(curr_gene)
        if is_chip_gene(curr_gene, curr_var):
            return True
    return False



#### Get dendro cluster and mean af for each donor and concat

In [10]:
clone_som_d = {}
clone_som_vars_d = {}

# for c in conditions:
#     try:
#         print(c)
#         clone_som_d[c] = pd.read_csv(join(indir, c, "clones_alt_numCells.csv"), index_col=0)
#         clone_som_vars_d[c] = pd.read_csv(join(indir, c, "clone_alt_numCells.variants.tsv"), sep="\t")
#         clone_som_d[c]["condition"] = c
#         clone_som_d[c]["name"] = clone_som_d[c].index
#     except:
#         print(f"{c} not here")

for c in conditions:
    print(c)
    if not exists(join(indir, c, "clones_alt_numCells.csv")):
        print(f"{c} numCells not here")
    elif not exists(join(indir, c, "clone_alt_numCells.variants.tsv")):
        print(f"{c} numCell variants not here")    
    else:
        clone_som_d[c] = pd.read_csv(join(indir, c, "clones_alt_numCells.csv"), index_col=0)
        clone_som_vars_d[c] = pd.read_csv(join(indir, c, "clone_alt_numCells.variants.tsv"), sep="\t")
        clone_som_d[c]["condition"] = c
        clone_som_d[c]["name"] = clone_som_d[c].index


In [11]:
clone_som_df = pd.concat(clone_som_d, axis=0).reset_index(drop=True).fillna(0)
clone_som_merge_df = clone_som_df.drop("condition",axis=1).groupby("name").agg(sum).astype(int)
clone_som_merge_df


## Process the variants 
clone_som_vars_df = pd.concat(clone_som_vars_d) #[conditions[0]].reset_index()
# for c in conditions[1:]:
#     clone_som_vars_df = pd.merge(clone_som_vars_df, clone_som_vars_d[c].reset_index(), how="outer" ) #, left_index=True, right_index=True, 
clone_som_vars_df = clone_som_vars_df.drop_duplicates()
clone_som_vars_df

var                        Gene Name Feature_type  \
Control 0    chr10_100267146_C                          CWF19L1   Transcript   
        1    chr10_102504349_T                      SUFU;ACTR1A   Transcript   
        2    chr10_102646092_A            ENSG00000272933;TRIM8   Transcript   
        3    chr10_102963521_G                            CNNM2   Transcript   
        4    chr10_110211097_A             ENSG00000228417;MXI1   Transcript   
...                        ...                              ...          ...   
Input   765    chrY_11333113_A  ENSG00000258991;ENSG00000270455   Transcript   
        766    chrY_11333831_T  ENSG00000258991;ENSG00000270455   Transcript   
        767    chrY_11334230_T  ENSG00000258991;ENSG00000270455   Transcript   
        768    chrY_11334344_A  ENSG00000258991;ENSG00000270455   Transcript   
        769    chrY_11334378_A  ENSG00000258991;ENSG00000270455   Transcript   

                                                   Consequence  \
Control 0    intron_variant,non_coding_transcript_variant;i...   
        1                 upstream_gene_variant;intron_variant   
        2    upstream_gene_variant;intron_variant,non_codin...   
        3                                       intron_variant   
        4                 upstream_gene_variant;intron_variant   
...                                                        ...   
Input   765  non_coding_transcript_exon_variant;upstream_ge...   
        766      downstream_gene_variant;upstream_gene_variant   
        767      downstream_gene_variant;upstream_gene_variant   
        768      downstream_gene_variant;upstream_gene_variant   
        769      downstream_gene_variant;upstream_gene_variant   

                       posID ref Allele  
Control 0    chr10_100267146   G      C  
        1    chr10_102504349   G      T  
        2    chr10_102646092   C      A  
        3    chr10_102963521   T      G  
        4    chr10_110211097   C      A  
...                      ...  ..    ...  
Input   765    chrY_11333113   G      A  
        766    chrY_11333831   G      T  
        767    chrY_11334230   A      T  
        768    chrY_11334344   C      A  
        769    chrY_11334378   C      A  

[1519 rows x 7 columns]

In [15]:
def add_chr_pos(x):
    spl = x.name.split("_")
    return [spl[0], spl[1]]


clone_som_vars_df = clone_som_vars_df.set_index("posID")
clone_som_vars_df[["chr", "pos"]] = clone_som_vars_df.parallel_apply(add_chr_pos , axis=1, result_type="expand")




In [16]:
clone_som_vars_df["pos"] =  clone_som_vars_df["pos"].astype(int)

In [17]:
if to_chip:
    clone_som_vars_df["is_chip"] = clone_som_vars_df.apply(is_in_any_chip_gene, args=(chip_df,), axis=1)
    clone_som_vars_df = clone_som_vars_df[clone_som_vars_df["is_chip"]].set_index("var")

clone_som_vars_df["Gene Name"] = clone_som_vars_df["Gene Name"].fillna("")
clone_som_vars_df["Gene Name"] = clone_som_vars_df.apply(lambda x: x["Gene"] if x['Gene Name']=="" else x['Gene Name'] , axis=1)


clone_som_vars_df = clone_som_vars_df.reset_index().drop_duplicates(subset=["var", "Gene Name", "Feature_type", "Consequence"]).sort_values("Gene Name").iloc[::-1]


In [18]:
## For duplicated variants, add to gene
def dups(df):
    if df.shape[0] == 0:
        return df.iloc[0]

    cons = ";".join(list(set(df["Consequence"].values)))
    gene_name = ";".join(list(set(df["Gene Name"].values)))
    out = df.iloc[0]
    out["Consequence"] = cons
    out["Gene Name"] = gene_name
    out.name = None
    return out

clone_som_vars_df_rmdups = clone_som_vars_df.groupby("var").apply(dups)
clone_som_vars_df_rmdups = clone_som_vars_df_rmdups[["Gene Name", "Feature_type", "Consequence"]]



if to_chip:
    clone_som_merge_df = clone_som_merge_df.loc[:, clone_som_vars_df_rmdups.index]



In [19]:
donors = cells_meta["donor"].unique()
conditions = cells_meta["condition"].unique()

clones = []
for d in donors:
    clones.append(pd.read_csv(join(dendro_indir, f"donor{d}.clones_dendro.csv"), index_col=0))
    clones[-1]["den_clust"] = str(d) + "_" + clones[-1]["den_clust"] 
clones = pd.concat(clones)

# mean_af = pd.concat(mean_af, axis=0, ignore_index=False).fillna(0)
# mean_af

## Split by donor

# Calculate enrichment for each clone
A. Fisher's exact test for variant in clone vs not (account for clone sizes)  
B. Remove variants seen in >50% of clones    
C. Entropy for the variant num cells in each clone  
C. Rank by (%Cells in clone with variant)/(%cells not in clone with variant) .. calculate percent by adding pseudocount of 1 to both


In [20]:
from scipy.stats import fisher_exact

def fish_test(variant_ser, clone_id, clones_meta):
    x_pos = variant_ser.loc[clone_id]

    x_neg = clones_meta.loc[clone_id, 'size'] - x_pos
    
    y_pos = sum(variant_ser.loc[variant_ser.index!=clone_id])
    y_neg = clones_meta['size'].sum() - x_pos - x_neg - y_pos
#     print('clone_id', clone_id)
#     print('var', variant_ser.name)
#     print(f"cells in clone with var: {x_pos} ; cells in clone: {x_neg}")
#     print(f"cells not in clone with var: {y_pos};  cells not in clone: {y_neg} ")
    oddsratio, pval = fisher_exact([[x_pos,x_neg],[y_pos,y_neg]], alternative='greater')
    return pval

def clone_fish(var_ser, clones_meta):
    #print(var_ser)
    pval_ser = pd.Series(index=var_ser.index, dtype=float)
    for clone_id in var_ser.index:
        pval_ser.loc[clone_id] = fish_test(var_ser, clone_id, clones_meta)
    #print(pval_ser)
    return pval_ser.fillna(1)

def var_fish(df, clones_meta):
    pval_df = df.apply(clone_fish, args=(clones_meta,), axis=0)
    return pval_df

def get_sig_var_clone(sig_inds, pval_df, ncells_var_df, curr_cl):
    """ Filter for significant p-values and add number of cells information 
    """
    sig_var_clone = []
    clones = pval_df.index
    variants = pval_df.columns
    for cl_ind, vrt_ind in sig_inds:
        cl = clones[cl_ind]
        vrt = variants[vrt_ind]
        ncells_cl = curr_cl.loc[cl, "size"]
        ncells_var_cl = ncells_var_df.loc[cl, vrt]
        pct = ncells_var_cl/ncells_cl
        
        ncells_oth_cl = curr_cl["size"].sum() - ncells_cl
        ncells_var_oth_cl = sum(ncells_var_df[vrt]) - ncells_var_cl
        pct_oth = ncells_var_oth_cl/ncells_oth_cl
        
        var_pct = ncells_var_cl/(ncells_var_cl+ncells_var_oth_cl)
        
        diff = pct-pct_oth
        
        sig_var_clone.append((cl, vrt, pval_df.loc[cl, vrt], var_pct, diff,
                              pct, ncells_var_cl, ncells_cl, 
                              pct_oth, ncells_var_oth_cl, ncells_oth_cl)) #, curr_cl)
        
        
    return pd.DataFrame(sig_var_clone, columns=["clone", "variant", "Fisher's exact pval", 
                                                "var_pct_in_clone","diff",
                                                "ncells_pct", "ncells_var_cl", "ncells_cl", 
                                                "ncells_oth_pct", "ncells_var_oth", "ncells_oth" ])



In [21]:
def get_top_clone_vars_donor(curr_som, curr_cl):
    pval_df = var_fish(curr_som, curr_cl)
    sig_inds = np.argwhere(pval_df.values<0.05)

    sig_var_clone = get_sig_var_clone(sig_inds, pval_df, curr_som, curr_cl)
    sig_var_clone

    ### Filter (add column) where ncells oth clones is high (>100)

    sig_var_clone["oth_100"] = sig_var_clone["ncells_var_oth"]>100
    sum(sig_var_clone["oth_100"])

    sig_var_clone_nooth = sig_var_clone.loc[~(sig_var_clone["oth_100"])]
    sig_var_clone_nooth.sort_values(["var_pct_in_clone","ncells_var_cl"])[::-1].head(10)

    sig_var_clone_nooth = sig_var_clone.loc[~(sig_var_clone["oth_100"])]
    sig_var_clone_nooth.sort_values(["ncells_var_cl"])[::-1].head(10)

    return sig_var_clone_nooth

In [22]:
def get_top_dendro_vars_donor(curr_som, curr_cl):
    ## Do for dendro
    den_som_df = pd.concat((curr_som, curr_cl[["den_clust"]]), axis=1, ignore_index=False).groupby("den_clust").agg(sum)
    den_som_df

    den_clone_df = pd.DataFrame(curr_cl.groupby("den_clust").sum()["size"])
    den_clone_df


    den_pval_df = var_fish(den_som_df, den_clone_df)
    sig_inds = np.argwhere(den_pval_df.values<0.05)

    sig_var_dendro = get_sig_var_clone(sig_inds, den_pval_df, den_som_df, den_clone_df)
    sig_var_dendro

    ### Filter (add column) where ncells oth dendros is high (>100)

    sig_var_dendro["oth_100"] = sig_var_dendro["ncells_var_oth"]>100
    sum(sig_var_dendro["oth_100"])

    sig_var_dendro_nooth = sig_var_dendro.loc[~(sig_var_dendro["oth_100"])]
    sig_var_dendro_nooth.sort_values(["diff"])[::-1].head(10)
    return sig_var_dendro_nooth

In [23]:
donors_clones = {}
donors_dendro = {}

for d, curr_cl in clones.groupby("donor"):
    print('d', d)
    curr_som = clone_som_merge_df.loc[clone_som_merge_df.index.isin(curr_cl.index)]
    curr_som = curr_som.loc[:, ~((curr_som==0).all())]
    print('curr_som', curr_som.shape)
    donors_clones[d] = get_top_clone_vars_donor(curr_som, curr_cl)
    donors_dendro[d] = get_top_dendro_vars_donor(curr_som, curr_cl)
#     if curr_som.shape[0] == 1:
#         f = plt.figure()
#         sns.heatmap(np.log10(curr_som+1))
#     else:
#         ch.plot_cluster(np.log10(curr_som+1), row_meta=curr_cl.drop("donor",axis=1), 
#                    row_clr_schemes={"size":"sequential", "den_clust": "categorical", "donor":"categorical"})

#     plt.suptitle(f"Donor {d}")
    #fu.helper_save(join(outdir, f"donor{d}_som_vars_dendro.png"), to_pdf=False)
    #fu.helper_save(join(outdir, f"donor{d}_som_vars_dendro.png"), to_pdf=False)

d 0
curr_som (34, 161)
d 1
curr_som (33, 162)


In [29]:
clone_som_vars_df_rmdups

,Gene Name,Feature_type,Consequence
var,,,
chr11_1016905_T,AP2A2;MUC6;MUC6;AP2A2,Transcript,missense_variant;upstream_gene_variant;downstr...
chr11_10607775_T,LYVE1;IRAG1;IRAG1;LYVE1,Transcript,"intron_variant,non_coding_transcript_variant;d..."
chr11_108120896_G,ENSG00000255467;ACAT1;ACAT1;ENSG00000255467,Transcript,"intron_variant,non_coding_transcript_variant;i..."
chr11_122506084_T,ENSG00000255248,Transcript,"intron_variant,non_coding_transcript_variant"
chr11_126586049_C,ENSG00000257271;KIRREL3;KIRREL3;ENSG00000257271,Transcript,"intron_variant,non_coding_transcript_variant;i..."
...,...,...,...
chr4_74189490_C,MTHFD2L,Transcript,intron_variant;upstream_gene_variant;intron_va...
chr4_75672581_C,G3BP2,Transcript,"intron_variant,non_coding_transcript_variant;i..."
chr4_82892864_A,SEC31A;ENSG00000251022;ENSG00000251022;SEC31A,Transcript,downstream_gene_variant;intron_variant;upstrea...


In [33]:
donors_dendro_df = pd.concat(donors_dendro).reset_index().rename({"level_0":"donor"},axis=1)
donors_dendro_df
donors_dendro_df = pd.merge(donors_dendro_df,clone_som_vars_df_rmdups, left_on="variant", right_index=True)
donors_dendro_df.sort_values([ "var_pct_in_clone", "ncells_var_cl"], ascending=False).to_csv(join(outdir,"top_dendro_somvars.csv"),
                                                                                             index=False)

In [36]:
donors_clones_df = pd.concat(donors_clones).reset_index(0).rename({"level_0":"donor"},axis=1)
donors_clones_df = pd.merge(donors_clones_df,clone_som_vars_df_rmdups, left_on="variant", right_index=True)
donors_clones_df.sort_values(["var_pct_in_clone", "Fisher's exact pval", "diff"], ascending=[False, True, False])

donors_clones_df.to_csv(join(outdir,"top_clone_somvars.csv"),index=False)

In [26]:
outdir

'/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/somatic_variants/gatk_mutect/peaks_atac_peaks/som_dendro_0.6/chip_clones_variants_summary'